# model io
## fewshot prompt template
- 퓨샷은 모델에게 예제를 주어준다는 뜻이다. 더 나은 대답을 위한 예제를 주는 편이 어떻게 대답해야 알려주는 것보다 좋다.
- '컴마를 써서 구분해줘, 소문자로 만들어줘.' 라는 말보다 어떤 결과를 원하는지 보여주자.


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Germany")
chain = prompt | chat
chain.invoke({
    "country": "Germany"
})

AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

## few shot chat message prompt template

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
{
"country": "France",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"country": "Italy",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"country": "Greece",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give a short answer"),
    example_prompt,
    ("human", "What do you know about {country}?")
])

# prompt.format(country="Germany")
chain = final_prompt | chat
chain.invoke({
    "country": "Germany"
})

I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='I know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

## length based example selector

- 다이나믹 셀렉터로 많은 예제 중에 특정한 예제를 골라서 프롬프트에 입력할 수 있다. 프롬프트가 길수록 가격이 많이 나오기 때문이다.
- 정해 놓은 예제의 길이따라 알맞은 프롬프트 예제를 고른다.

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

'\n    Human: What do you know about France?\n    AI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\n\n\n    Human: What do you know about Italy?\n    AI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\n\n    Human: What do you know about Greece?\n    AI: \nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\n\nHuman: What do you know about Germany?'

## random example selector
- 랜덤으로 예제를 하나 선택한다.


In [11]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples
    
    # 나만의 랜덤 예제 선택기를 만들려면 이 메소드가 필요하다.
    # 이미 존재하는 examples에 example를 추가하는 메소드다.
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

'\n    Human: What do you know about Italy?\n    AI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\nHuman: What do you know about Germany?'

## serialization and composition

- 프롬프트를 저장하고 불러올 때 직렬화를 해야한다.
- 보통 프롬프트는 yaml이나 json 형식으로 불러와서 사용한다.

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt


prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

prompt.format(country="Germany")


'What is the capital of Germany'

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

prompt.format(country="Germany")


'What is the capital of Germany'

## PipelinePromptTemplate
- 프롬프트 파일들을 하나의 파이프라인으로 합칠 수 있다.
- 역할을 주고 어떻게 질문하고 대답할지 예시를 주는 과정이다.

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}.
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
    {example}
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# full_prompt.format(
#     character="Priate", 
#     example_question="What is your location?", 
#     example_answer="Arrg! That is a secret! Arg arg!",
#     question="What is your fav food?"    
# )

chain = full_prompt | chat

chain.invoke(
    {
        "character":"Priate", 
        "example_question":"What is your location?", 
        "example_answer":"Arrg! That is a secret! Arg arg!",
        "question":"What is your fav food?"
    }
)

Arrg! Me favorite food be a good ol' plate o' fish and chips! Arg arg!

AIMessageChunk(content="Arrg! Me favorite food be a good ol' plate o' fish and chips! Arg arg!")

## caching

- 자주 요청하는 프롬프트를 메모리에 캐싱해서 빠른 응답을 할 수 있다.
- SQLiteCache 디비로 프롬프트 메세지를 캐싱할 수 있다.
- 서드파티 db를 이용할 수 있다.

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
)



In [12]:
chat.predict("How do you make italian pasta?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too sticky, add a little more 

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too sticky, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough

## serialization
- `get_openai_callback` openai api 비용이 얼마나 사용되는지 알 수 있다.

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)
set_debug(False)
with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for ramen?")
    b = chat.predict("What is the recipe for soju?")
    print(a,b,"\n")
    print(usage)


Ramen is a popular Japanese noodle dish that typically consists of four main components: broth, noodles, toppings, and seasonings. Here's a basic recipe for ramen:

Ingredients:
- 4 cups of chicken or vegetable broth
- 2 packs of ramen noodles (discard the seasoning packets)
- 2 cups of water
- 2 tablespoons of soy sauce
- 1 tablespoon of miso paste (optional)
- 1 tablespoon of sesame oil
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- Toppings of your choice (e.g., sliced cooked pork, soft-boiled eggs, sliced green onions, seaweed, corn, mushrooms, etc.)

Instructions:
1. In a large pot, heat the sesame oil over medium heat. Add the minced garlic and grated ginger, and sauté for a minute until fragrant.
2. Pour in the chicken or vegetable broth and water. Bring it to a boil.
3. Once boiling, reduce the heat to low and let it simmer for about 15-20 minutes to allow the flavors to meld together.
4. While the broth simmers, cook the ramen noodles according to the package 

In [19]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
)

chat.save("model.json")

AttributeError: 'ChatOpenAI' object has no attribute 'save'